In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from importlib import reload

import train as t
import model
import loss
import utils

In [2]:
reload(utils)

<module 'utils' from '/users/invites/jmorlot/Documents/Projects/Seq2Seq_Nucleosom/utils.py'>

In [3]:
reload(model)

<module 'model' from '/users/invites/jmorlot/Documents/Projects/Seq2Seq_Nucleosom/model.py'>

In [4]:
reload(t)

<module 'train' from '/users/invites/jmorlot/Documents/Projects/Seq2Seq_Nucleosom/train.py'>

In [ ]:
s2s = t.Seq2Seq_model(fasta_path='Nucleosom_data/Reference sequence-chrI-1..230218.fasta',
                      histone_path='Nucleosom_data/Chereji_2018_Occupancy_H3_CC_V64-chrI-1..230218.bedgraph',
                      output_size=1,
                      Nepochs=10,
                      seq_len=5000,
                      batchsize=1,
                      hidden_size=8,
                      input_dropout_p=0,
                      n_layers=1,
                      use_cuda=True)

In [ ]:
s2s.fit()

train:   0%|          | 0/36 [00:00<?, ?it/s]